# Cyclistic Case Study
#### Author: Danny Lam
#### Date: 10/1/2021

### Case Background
Cyclistic is a bike-sharing company in Chicago with 5,824 geotracted bicycles and 692 docking stations across the city. As a mock data analyst working in the marketing analyst team at Cyclistic (a bike-sharing company active in Chicago), I have been tasked with understanding how casual riders and annual members use Cyclistic bikes differently. Casual riders are made up of customers that who purchase single-ride or full-day passes, whereas annual members purchase an annual subscription for the service. The director of marketing theorizes that the company's future success depends on maximizing the number of annual memberships. Pending executive approval, my team is planning on designing a new marketing strategy to convert casual users into annual subscribers.

The goal of this case study is to uncover and convey actionable insights to inform executive decision-making behind the future of Cyclistic's marketing program.

In [20]:
#import libraries
import numpy as np                        # for efficient data types
import csv                                # to take a peek at our CSV files
import os                                 
import glob                               # pathnames matching
import pandas as pd                       # data manipulation and analysis with parallelization 
import warnings                           # to get rid of markdown warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt           # to support plotting visuals
import seaborn as sns                     # visualization module
import difflib                            # for comparing strings
import re                                 # for pattern matching 

#### Import data gathered and provided to me from Cyclistic 
- The data we'll be using was extracted from [here](https://divvy-tripdata.s3.amazonaws.com/index.html) and is made available by Motivate International Inc. under this [license](https://www.divvybikes.com/data-license-agreement). 
- Note that Cyclistic is a fictional entity and Divvy's open data is used for the purpose of this case study.
- The data includes the most recent 12 months available, ranging from October 2020 to September 2021.
- There are 5.1 million recorded bike rides over the year.

In [44]:
# merging 12 csv files
joined_files = os.path.join("CSV files/", "202*.csv")
  
# list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# finally, the files are joined and stored in bike_data dataframe
bike_data = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)


In [45]:
# get general info of data
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136261 entries, 0 to 5136260
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
 13  ride_length         object 
 14  day_of_week         int64  
dtypes: float64(4), int64(1), object(10)
memory usage: 587.8+ MB


#### Since the columns 'started_at', 'ended_at', and 'ride_length' are object dtype, I will need change the format to datetime

In [60]:
# change the dtype to datetime
bike_data['started_at'] = pd.to_datetime(bike_data['started_at'], format = "%m/%d/%y %H:%M" )

bike_data['ended_at'] = pd.to_datetime(bike_data['ended_at'], format = "%m/%d/%y %H:%M" )

In [67]:
# set 'ride_length' by subtracting 'started_at' from 'ended_at' and it'll convert the dtype to timedelta64
bike_data['ride_length'] = bike_data['ended_at'] - bike_data['started_at']

In [70]:
# now checking to see if the dtype changed to datetime64
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136261 entries, 0 to 5136260
Data columns (total 15 columns):
 #   Column              Dtype          
---  ------              -----          
 0   ride_id             object         
 1   rideable_type       object         
 2   started_at          datetime64[ns] 
 3   ended_at            datetime64[ns] 
 4   start_station_name  object         
 5   start_station_id    object         
 6   end_station_name    object         
 7   end_station_id      object         
 8   start_lat           float64        
 9   start_lng           float64        
 10  end_lat             float64        
 11  end_lng             float64        
 12  member_casual       object         
 13  ride_length         timedelta64[ns]
 14  day_of_week         int64          
dtypes: datetime64[ns](2), float64(4), int64(1), object(7), timedelta64[ns](1)
memory usage: 587.8+ MB


#### Now the dtype for 'started_at' and 'ended_at' are datetime64 format

In [76]:
# let's check the first few rows of our data
bike_data.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week
0,BD0A6FF6FFF9B921,electric_bike,2020-11-01 13:36:00,2020-11-01 13:45:00,Dearborn St & Erie St,110,St. Clair St & Erie St,211,41.894177,-87.629127,41.894434,-87.623379,casual,00:09:00,1
1,96A7A7A4BDE4F82D,electric_bike,2020-11-01 10:03:00,2020-11-01 10:14:00,Franklin St & Illinois St,672,Noble St & Milwaukee Ave,29,41.890959,-87.635343,41.900675,-87.662480,casual,00:11:00,1
2,C61526D06582BDC5,electric_bike,2020-11-01 00:34:00,2020-11-01 01:03:00,Lake Shore Dr & Monroe St,76,Federal St & Polk St,41,41.880983,-87.616754,41.872054,-87.629550,casual,00:29:00,1
3,E533E89C32080B9E,electric_bike,2020-11-01 00:45:00,2020-11-01 00:54:00,Leavitt St & Chicago Ave,659,Stave St & Armitage Ave,185,41.895499,-87.682013,41.917744,-87.691392,casual,00:09:00,1
4,1C9F4EF18C168C60,electric_bike,2020-11-01 15:43:00,2020-11-01 16:16:00,Buckingham Fountain,2,Buckingham Fountain,2,41.876497,-87.620358,41.876448,-87.620338,casual,00:33:00,1
5,7259585D8276D338,electric_bike,2020-11-14 15:55:00,2020-11-14 16:44:00,Wabash Ave & 16th St,72,Lake Shore Dr & Monroe St,76,41.860289,-87.625806,41.880985,-87.616773,casual,00:49:00,7
6,91FE5C8F8A676594,electric_bike,2020-11-14 16:47:00,2020-11-14 17:03:00,Lake Shore Dr & Monroe St,76,Wabash Ave & 16th St,72,41.881006,-87.616776,41.860474,-87.625842,casual,00:16:00,7
7,9E7A79ADA90C2695,electric_bike,2020-11-14 16:04:00,2020-11-14 16:19:00,NaN,NaN,NaN,NaN,41.910000,-87.620000,41.910000,-87.620000,casual,00:15:00,7
8,A5B02C0D41DBCDAF,electric_bike,2020-11-14 16:24:00,2020-11-14 16:51:00,Marshfield Ave & Cortland St,58,Larrabee St & Armitage Ave,288,41.916067,-87.669042,41.918149,-87.643875,casual,00:27:00,7
9,8234407C29FE41DC,electric_bike,2020-11-14 01:24:00,2020-11-14 01:31:00,Clark St & 9th St (AMLI),394,Michigan Ave & 18th St,273,41.870854,-87.631169,41.857912,-87.624667,casual,00:07:00,7


In [78]:
# get descriptive statistics under each numeric column
bike_data.describe()

,start_lat,start_lng,end_lat,end_lng,ride_length,day_of_week
count,5.136261e+06,5.136261e+06,5.131440e+06,5.131440e+06,5136261,5.136261e+06
mean,4.190240e+01,-8.764559e+01,4.190269e+01,-8.764581e+01,0 days 00:20:37.426404,4.125390e+00
std,4.489459e-02,2.742823e-02,4.499254e-02,2.759811e-02,0 days 05:03:18.822410,2.081543e+00
min,4.164000e+01,-8.784000e+01,4.151000e+01,-8.807000e+01,-21 days +19:50:00,1.000000e+00
25%,4.188213e+01,-8.765975e+01,4.188224e+01,-8.766000e+01,0 days 00:07:00,2.000000e+00
50%,4.190000e+01,-8.764170e+01,4.190000e+01,-8.764182e+01,0 days 00:13:00,4.000000e+00
75%,4.192947e+01,-8.762773e+01,4.192955e+01,-8.762775e+01,0 days 00:23:00,6.000000e+00
max,4.208000e+01,-8.752000e+01,4.216812e+01,-8.744000e+01,38 days 20:24:00,7.000000e+00
